In [1]:
from absl import logging
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
from seaborn import color_palette
import cv2
import pandas as pd
from tqdm import tqdm
import os
import time
import datetime
import csv
from sklearn.model_selection import StratifiedKFold
# from my_yolo import *

In [2]:
images = os.listdir(r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\malaria\images") # read the images data where all images are present
train = pd.read_json(r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\malaria\training.json") # read the train json file where info about all images
# are given including bounding box information.
test = pd.read_json(r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\malaria\test.json") # read the test json file.

In [ ]:
# here we have store train and test image in different list
train_image = [] # train images stored here
test_image = [] # test images stored here
for df in train['image']:
    train_image.append(df['pathname'].split('/')[-1])
for df in test['image']:
    test_image.append(df['pathname'].split('/')[-1])
print("Train image length is -> ",len(train_image))
print("Train image :",train_image[:5])
print("Test image length is -> ",len(test_image))
print("Test image :",test_image[:5])

class_dict = {'red blood cell':0,'trophozoite': 1, 'schizont': 2, 'difficult': 3, 'ring': 4,
              'leukocyte': 5, 'gametocyte': 6}

#class_dict = {'uninfected': 0, 'infected': 1}
# we have defined the dictionary for labels or classes.

In [4]:
# Define output directories
train_output_dir = r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SETs\train_images"
test_output_dir = r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SETs\test_images"

# Create directories if they don't exist
os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(test_output_dir, exist_ok=True)

# Save train images
for image_name in train_image:
    src_path = os.path.join(r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\malaria\images", image_name)
    dest_path = os.path.join(train_output_dir, image_name)
    if os.path.exists(src_path):
        Image.open(src_path).save(dest_path)

# Save test images
for image_name in test_image:
    src_path = os.path.join(r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\malaria\images", image_name)
    dest_path = os.path.join(test_output_dir, image_name)
    if os.path.exists(src_path):
        Image.open(src_path).save(dest_path)

class_dict = {'red blood cell':0,'trophozoite': 1, 'schizont': 2, 'difficult': 3, 'ring': 4,
              'leukocyte': 5, 'gametocyte': 6}

#class_dict = {'uninfected': 0, 'infected': 1}
# we have defined the dictionary for labels or classes.

In [ ]:
# Define base paths for train and test images
train_base_path = r'C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\malaria\train_images'
test_base_path = r'C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\malaria\test_images'

# Define output directory
output_dir = r'C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SETs'
os.makedirs(output_dir, exist_ok=True)

# Function to extract image info and save to separate files
def extract_and_save_image_info(data, base_path, filename):
    info_list = []
    for _, row in data.iterrows():
        path = os.path.join(base_path, row['image']['pathname'].replace('/', '\\'))
        for info in row['objects']:
            category = info['category']  # Category as label
            bounding_box = info['bounding_box']
            box_dim = (
                bounding_box['maximum']['r'], bounding_box['maximum']['c'],
                bounding_box['minimum']['r'], bounding_box['minimum']['c'],
                category, path
            )
            info_list.append(box_dim)
    
    # Save to separate TXT files
    headers = ['max_r', 'max_c', 'min_r', 'min_c', 'category', 'img_path']
    file_path = os.path.join(output_dir, filename)
    with open(file_path, 'w') as f:
        f.write('\t'.join(headers) + '\n')  # Write headers
        for row in info_list:
            f.write('\t'.join(map(str, row)) + '\n')  # Write data

    return file_path

# Save train and test data separately
train_txt_file = extract_and_save_image_info(train, train_base_path, 'train_data.txt')
test_txt_file = extract_and_save_image_info(test, test_base_path, 'test_data.txt')

# Load TXT files into pandas DataFrames
df_train = pd.read_csv(train_txt_file, sep='\t')
df_test = pd.read_csv(test_txt_file, sep='\t')

print(f"Train data contains {df_train.shape[0]} rows and {df_train.shape[1]} columns.")
print(f"Test data contains {df_test.shape[0]} rows and {df_test.shape[1]} columns.")

# Check unique image count
unique_images_train = df_train['img_path'].nunique()
unique_images_test = df_test['img_path'].nunique()

print(f"Unique train images: {unique_images_train}")
print(f"Unique test images: {unique_images_test}")

# Map category labels
df_train['label'] = df_train['category'].map(class_dict)
df_test['label'] = df_test['category'].map(class_dict)

data_train = df_train
data_test = df_test